In [83]:
import pandas as pd
pd.set_option("display.max_seq_items", None)  
pd.set_option("display.width", None)         

In [84]:
raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")

C:\Users\alexi\AppData\Local\Temp\ipykernel_15876\2691490454.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")


In [85]:
data = raw_data.copy()

In [86]:
# for 2005 to 2025, read pickle file get gvkeys into a dictionary and delete the dataframe to free up memory
gvkeys_dict = {}
for year in range(2005, 2026):
    text_data = pd.read_pickle(f"C:\\Users\\alexi\\OneDrive\\Documents\\école\\McGill-FIAM\\2025\\Hackathon-Final-2025\\DATA ASSET MANAGEMENT HACKATHON 2025 FINALS\\TEXT DATA US by YEAR\\{year}\\text_us_{year}.pkl")
    gvkeys_dict[year] = text_data['gvkey'].unique().tolist()
    del text_data

In [87]:
# For every gvkey in the dictionary, filter the data dataframe to only include rows with those gvkeys for the corresponding year
filtered_data_list = []
for year in range(2005, 2026):
    # data from that year contains only gvkeys in gvkeys_dict[year]
    filtered_data = data[(data['year'] == year) & (data['gvkey'].isin(gvkeys_dict[year]))]
    filtered_data_list.append(filtered_data)

# concatenate all filtered dataframes
filtered_data = pd.concat(filtered_data_list, ignore_index=True)
del filtered_data_list

In [88]:
# Keep columns: date, excntry, stock_ret, year, month, char_date, market_equity, be_me, ni_me, at_gr1, tangibility, at_be, debt_me, div12m_me, eqpo_me, eqnetis_at, debt_iss, ni_be, profit_sale, gp_at, turnover_126d
filtered_data = filtered_data[['date', 'gvkey', 'excntry', 'stock_ret', 'year', 'month', 'char_date', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d']]

In [89]:
joining_table = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/North America Company Name Merge by DataDate-GVKEY-IID.csv")
# rename datadate to date
joining_table = joining_table.rename(columns={"datadate": "date"})

In [90]:
# --- Keys côté filtered_data ---
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['char_date_key'] = pd.to_datetime(filtered_data['char_date'].astype(str), format='%Y%m%d', errors='coerce')

# --- Keys côté joining_table ---
joining_table['gvkey_key'] = pd.to_numeric(joining_table['gvkey'], errors='coerce').astype('Int64')
joining_table['char_date_key'] = pd.to_datetime(joining_table['date'], errors='coerce')

# --- Merge sur gvkey + char_date ---
filtered_data = filtered_data.merge(
    joining_table[['gvkey_key', 'char_date_key', 'tic', 'conm']],
    on=['gvkey_key', 'char_date_key'],
    how='left'
)

In [91]:
sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")

C:\Users\alexi\AppData\Local\Temp\ipykernel_15876\3118638869.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")


In [92]:
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['date_key']  = pd.to_datetime(filtered_data['date'].astype(str), format='%Y%m%d', errors='coerce')

# --- Keys sector_mapping (date = YYYYMMDD en int) ---
sm = sector_mapping[['gvkey','date','gics','sic','naics']].copy()
sm['gvkey_key'] = pd.to_numeric(sm['gvkey'], errors='coerce').astype('Int64')
sm['date_key']  = pd.to_datetime(sm['date'].astype(str), format='%Y%m%d', errors='coerce')

# (optionnel) éviter les duplications si plusieurs lignes par gvkey-date
sm = sm.drop_duplicates(subset=['gvkey_key','date_key'])

# --- Merge ---
filtered_data = filtered_data.merge(
    sm[['gvkey_key','date_key','gics','sic','naics']],
    on=['gvkey_key','date_key'],
    how='left'
).drop(columns=['gvkey_key','date_key'])

In [93]:
# 1) Extraire le code secteur GICS (2 premiers chiffres) à partir du code GICS 8 chiffres
# ex: 20101010 -> 20
filtered_data['gics_sector_code'] = (
    pd.to_numeric(filtered_data['gics'], errors='coerce')
      .floordiv(10**6)
      .astype('Int64')
)

# 2) Mapping MSCI / GICS (11 secteurs)
gics_sector_map = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",
    25: "Consumer Discretionary",
    30: "Consumer Staples",
    35: "Health Care",
    40: "Financials",
    45: "Information Technology",
    50: "Communication Services",
    55: "Utilities",
    60: "Real Estate",
}

# 3) Ajouter le nom du secteur
filtered_data['gics_sector_name'] = filtered_data['gics_sector_code'].map(gics_sector_map)
filtered_data

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,gp_at,turnover_126d,char_date_key,tic,conm,gics,sic,naics,gics_sector_code,gics_sector_name
0,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.132266,0.002816,2005-01-31,AL.1,ALCAN INC,15104010.0,3350.0,331319.0,15,Materials
1,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.132266,0.002816,2005-01-31,AL.Z,ALCAN INC,15104010.0,3350.0,331319.0,15,Materials
2,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
3,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY.,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
4,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY.PB,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657649,20250630,315318.0,USA,0.063407,2025,6,20250530,5184.735520,0.477729,0.046791,...,0.244835,0.010482,2025-05-30,NaN,NaN,15101050.0,NaN,NaN,15,Materials
657650,20250630,316056.0,USA,0.013758,2025,6,20250530,12279.192300,0.122215,0.048660,...,0.397656,0.011089,2025-05-30,NaN,NaN,20102010.0,NaN,NaN,20,Industrials
657651,20250630,317264.0,USA,0.138720,2025,6,20250530,913.093680,1.170207,0.176673,...,-0.067016,0.018638,2025-05-30,NaN,NaN,10102040.0,NaN,NaN,10,Energy
657652,20250630,326688.0,USA,0.113222,2025,6,20250530,10833.048800,0.321267,0.022228,...,0.201206,0.012945,2025-05-30,NaN,NaN,20104010.0,NaN,NaN,20,Industrials


In [94]:
filtered_data['tic'].unique()

array(['AL.1', 'AL.Z', 'NXY', ..., 'MRP', 'UMH.PD', 'FLYYQ'],
      shape=(6070,), dtype=object)

In [95]:
from pathlib import Path
import pandas as pd
import ast
import re

# Répertoire (relatif au notebook)
SP500_DIR = Path("sp500-master/sp500_constituants_2005_2024")

def norm_tic(s):
    if pd.isna(s):
        return pd.NA
    s = str(s).strip().upper()
    # optionnel: harmoniser BRK.B vs BRK-B, BF.B vs BF-B, etc.
    s = s.replace("-", ".")
    return s

# 1) Charger tous les fichiers annuels et construire (year, tic_norm)
members = []
for fp in sorted(SP500_DIR.glob("*-sp500-ticker-list.csv")):
    # année depuis le nom du fichier (ex: 2006-sp500-ticker-list.csv)
    year = int(fp.name.split("-")[0])

    df = pd.read_csv(fp)
    # tickers est une string du type "['A', 'AAPL', ...]"
    tickers = ast.literal_eval(df.loc[0, "tickers"]) if isinstance(df.loc[0, "tickers"], str) else df.loc[0, "tickers"]
    tickers = [norm_tic(t) for t in tickers]

    members.append(pd.DataFrame({"year": year, "tic_norm": tickers}))

sp500_members = pd.concat(members, ignore_index=True).dropna().drop_duplicates()

# 2) Filtrer filtered_data par année (S&P500 de l'année correspondante)
fd = filtered_data.copy()
fd["tic_norm"] = fd["tic"].map(norm_tic)

filtered_data = (
    fd.merge(sp500_members, on=["year", "tic_norm"], how="inner")
      .drop(columns=["tic_norm"])
)


In [96]:
# Forward fill values par gvkey, trié par date pour les colonnes: stock_ret, 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name'
filtered_data = filtered_data.sort_values(by=['gvkey', 'date'])
cols_to_ffill = ['stock_ret', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name']
filtered_data[cols_to_ffill] = (filtered_data.groupby('gvkey')[cols_to_ffill].ffill())

In [ ]:
# Réduction de la taille des données
# Garder données entre 2021 et 2023 inclus
# Éliminer la moitié des tickers

filtered_data = filtered_data[(filtered_data['year'] >= 2021) & (filtered_data['year'] <= 2023)]
filtered_data = filtered_data.sort_values(by=['gvkey'])
unique_gvkeys = filtered_data['gvkey'].unique()
reduced_gvkeys = unique_gvkeys[::2]  # garder un gvkey sur deux
filtered_data = filtered_data[filtered_data['gvkey'].isin(reduced_gvkeys)]

(5695, 30)

In [ ]:
# Afficher le nombre de tickers unique par année
for year in range(2005, 2026):
    n_unique_tickers = filtered_data[filtered_data['year'] == year]['gvkey'].nunique()
    print(f"Year {year}: {n_unique_tickers} unique tickers")

In [98]:
filtered_data.to_csv("yfinance/filtered_sp500_data.csv", index=False)